In [1070]:
import psycopg2
from psycopg2.extensions import AsIs
import psycopg2
import pandas as pd
import numpy as np

In [1071]:
%run -i ../sql_helpers.py
display(db_credentials)

{'dbname': 'gta',
 'port': 5432,
 'user': 'gta_p8',
 'password': 'r7sdkfdq',
 'host': 'ikgpgis.ethz.ch'}

In [1072]:
kirchen_zuerich = pd.read_csv("csv/kirchen-zurich.csv", encoding = "utf-8", sep="	")
restaurant_zurich = pd.read_csv("csv/restaurant-zurich.csv", encoding = "utf-8", sep="	")

In [1073]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [1074]:
sql_string_with_placeholders = "INSERT INTO church (address_id, church_name) VALUES (%s, %s)"
cur.mogrify(sql_string_with_placeholders,(100, "kirche"))

b"INSERT INTO church (address_id, church_name) VALUES (100, 'kirche')"

In [1075]:
conn.commit()
conn.close()

Kirchen

In [1076]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [1077]:
city_id_array = np.empty(0)

for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    #EXECUTE sql_commands:

    #city_table
    sql_fill_city = "INSERT INTO gta_p8.city (city_postcode, city_name) SELECT '"+city_postcode+"', '"+city_name+"' WHERE NOT EXISTS (select * from gta_p8.city where city_postcode='"+city_postcode+"');"

    cur.execute(sql_fill_city, (city_postcode, city_name))

    #address_table_preparation
    cur.execute("SELECT city_id FROM gta_p8.city WHERE city_postcode='"+city_postcode+"';")
    result = cur.fetchone()
    city_id_array = np.append(city_id_array, result[0])





In [1078]:
conn.commit()
conn.close()

In [1079]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [1080]:
address_id_array = np.empty(0)

for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>

    #EXECUTE sql_commands:

    #address_table
    city_id_fk = str(int(city_id_array[idx]))

    sql_fill_address = "INSERT INTO gta_p8.address (address_street, address_number, city_id) SELECT '"+address_street+"', '"+address_number+"', '"+city_id_fk+"' WHERE NOT EXISTS (select * from gta_p8.address where address_street='"+address_street+"' AND address_number='"+address_number+"');"
  
    #cur.execute(sql_fill_address, (address_street, address_number, city_id_fk))

    #kirchen_table_preparation
    cur.execute("SELECT address_id FROM gta_p8.address WHERE address_street='"+address_street+"' AND address_number='"+address_number+"';")
    result = cur.fetchone()
    address_id_array = np.append(address_id_array, result[0])

In [1081]:
conn.commit()
conn.close()

In [1082]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [1083]:
for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x
    if(type(church_name) == float): #nameless churches get skipped
        continue

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'


    #EXECUTE sql_commands:
    address_id_fk = str(int(address_id_array[idx]))

    sql_fill_church = "INSERT INTO gta_p8.church (church_name, address_id) SELECT '"+church_name+"', '"+address_id_fk+"' WHERE NOT EXISTS (select * from gta_p8.church where church_name='"+church_name+"');"

    cur.execute(sql_fill_church, (church_name, address_id_fk))

In [1084]:
conn.commit()
conn.close()

In [1085]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [1086]:
for idx, row in kirchen_zuerich.iterrows():
    #prepare geometry

    lon = str(row["@lon"]) #<class 'float'>

    lat = str(row["@lat"]) #<class 'float'>

    church_name = row['name'] #<class 'str'> x
    if(type(church_name) == float): #nameless churches get skipped
        continue

    geom_sql = "UPDATE gta_p8.church SET geometry = ST_SetSRID(ST_MakePoint("+lon+", "+lat+"),4326) WHERE church_name='"+church_name+"';"

    cur.execute(geom_sql)


In [1087]:
conn.commit()
conn.close()

## Restaurants

In [1088]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [1089]:
city_id_array = np.empty(0)

for idx, row in restaurant_zurich.iterrows():
    #prepare input values

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = str(int(city_postcode))

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'   

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>

    #EXECUTE sql_commands:

    #city_table
    sql_fill_city = "INSERT INTO gta_p8.city (city_postcode, city_name) SELECT '"+city_postcode+"', '"+city_name+"' WHERE NOT EXISTS (select * from gta_p8.city where city_postcode='"+city_postcode+"');"

    cur.execute(sql_fill_city, (city_postcode, city_name))

    #address_table_preparation
    cur.execute("SELECT city_id FROM gta_p8.city WHERE city_postcode='"+city_postcode+"';")
    result = cur.fetchone()
    city_id_array = np.append(city_id_array, result[0])
    

In [1090]:
conn.commit()
conn.close()

In [1091]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [1092]:
address_id_array = np.empty(0)

for idx, row in restaurant_zurich.iterrows():
    #prepare input values
    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    #EXECUTE sql_commands:

    #address_table
    city_id_fk = str(int(city_id_array[idx]))

    sql_fill_address = "INSERT INTO gta_p8.address (address_street, address_number, city_id) SELECT '"+address_street+"', '"+address_number+"', '"+city_id_fk+"' WHERE NOT EXISTS (select * from gta_p8.address where address_street='"+address_street+"' AND address_number='"+address_number+"');"
  
    cur.execute(sql_fill_address, (address_street, address_number, city_id_fk))

    #kirchen_table_preparation
    cur.execute("SELECT address_id FROM gta_p8.address WHERE address_street='"+address_street+"' AND address_number='"+address_number+"';")
    result = cur.fetchone()
    address_id_array = np.append(address_id_array, result[0])

In [1093]:
conn.commit()
conn.close()

In [1094]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [1095]:
for idx, row in restaurant_zurich.iterrows():
    #prepare input values
    restaurant_name = row['name'] #<class 'str'>
    if(type(restaurant_name) == float): #nameless restaurants get skipped
        continue

    restaurant_website = row['website'] #<class 'str'>
    if(type(restaurant_website) == float):
        restaurant_website = 'keine Website vorhanden!'

    restaurant_type = row['cuisine']
    if(type(restaurant_type) == float):
        restaurant_type = 'keine Kategorie!'

    #EXECUTE sql_commands:
    address_id_fk = str(int(address_id_array[idx]))

    sql_fill_restaurant = "INSERT INTO gta_p8.restaurant (address_id, restaurant_name, restaurant_website, restaurant_type) SELECT '"+address_id_fk+"', '"+restaurant_name+"', '"+restaurant_website+"', '"+restaurant_type+"' WHERE NOT EXISTS (select * from gta_p8.restaurant where restaurant_name='"+restaurant_name+"');"

    cur.execute(sql_fill_restaurant, (address_id_fk, restaurant_name, restaurant_website, restaurant_type))

In [1096]:
conn.commit()
conn.close()

In [1097]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [1098]:
for idx, row in restaurant_zurich.iterrows():
    #prepare input values
    restaurant_name = row['name'] #<class 'str'>
    if(type(restaurant_name) == float): #nameless restaurants get skipped
        continue   

    lon = str(row["@lon"]) #<class 'float'>

    lat = str(row["@lat"]) #<class 'float'>

    #EXECUTE sql_commands:

    geom_sql = "UPDATE gta_p8.restaurant SET geometry = ST_SetSRID(ST_MakePoint("+lon+", "+lat+"),4326) WHERE restaurant_name='"+restaurant_name+"';"

    cur.execute(geom_sql)

In [1099]:
conn.commit()
conn.close()